# Predicting *League of Legends* Ranked Victories by Early-Mid Game Statistics

## What is *League of Legends* and why are we interested in its victors?

*League of Legends* is a widely popular video game with a large folowing of their competitive scene and backed by the lore created by its company Riot Games that now spans four hugely popular games, multiple books, and a recent Netflix Series. All of this started as the creation of a video game in a then budding genre called Multiplayer Online Battle Arena, or MOBA. The MOBA genre has now exploded into becoming an over 300 million dollar industry with major e-sports contracts, worldwide competition, and events watched by more than some sports championships. *League of Legends* makes up two-thirds of the market share for this genre.

With this huge interest and low barrier of entry to the game, nearly every watcher has delved into playing at least a few games of League of Legends. And though some people are driven away by the toxic community, poor tutorials, or terrible networking code, there are multitudes more drawn in by the game's complexity, strategic depth, and competition. The game works on a ranked system, and in a world where time is money, people want to know whether a game is worth continuing or if it would be better to forfeit and move on to the next game. This brings us to our current question, predicting victors based on early game statistics.

## What early game statistics can we check?

Though *League of Legends* is a complex game with many strategic options that can even vary at the ranks of the competitive ladder, there is consistent things that are viewable at each and every rank. Though some of these features may vary in importance depending on rank, the presence and potential quantities of the features themselves is consistent due to the bounds of the game.

### Summary features
Each of these features are provided in a match summary format by the client. The featuers we have chosen are defined as being observable by a player on the loading screen. All three of these features is repeated once for each of the players for a total of ten times per feature.

#### Champion
The champion a player selects represents the character that they are using. This determines to a degree their playstyle for the match, the other features they may select, as well as the abilities available to them.

#### Summoner Spell
This feature represents one of the special skills taken by the player, called a summoner spell. Though players can select two of these spells, nearly all players select one called Flash and a secondary spell of their choice. Thus, to limit the amount of features, the flash summoner spell has been removed from the listing. A full description of each summoner spell and their uses can be found here: https://dignitas.gg/articles/blogs/Unknown/14425/an-overview-guide-to-summoner-spells-in-lol

#### Keystone Rune
As part of the game, each palyer creates what is known as a rune page. Though this can produce highly variable results with the amount of potential combinations, it is impossible to tell someone's full rune page until after the game has concluded. What you can easily see is the keystone rune selected by a given player. This is the rune with the most impact from those selected, and can drastically how a palyer wants to approach fighting against the player.


### Snapshot Features
These features are taken in three snapshots at five, ten, and fifteen minutes into the game. Each feature is duplicated, once to each team.

#### Team Gold
This is the total currency that the team has gained. Gold is generated passively as well as through various smaller and larger objectives. To ensure that there isn't too many features, the smaller features are lumped together in the total team gold. This can be considered the team's overall "score" in a sense.

#### Team Kills
This is the total amount of kills that a team has gotten. Though a kill does result in gold, it also results in taking a player out of the game temporarily and allowing the killing team to potentially take advantage of that player's temporary displacement.

#### Towers Destroyed
This is the amount of towers destroyed by the team. Towers are structures that shoot powerful projectiles at enemies, and are present in the open areas of the map. Though it is not common for many towers to be destroyed before the first fifteen minutes have elapsed, should one fall the players who have lost the tower will be losing significant amounts of control over that area of the map and allowing their enemies to apply much more pressure. See this article for a full explanation about League of Legends Towers, also known as Turrets: https://en.number13.de/league-of-legends-turret-aggro-and-everything-you-need-to-know-about-turrets/

#### Dragons Killed
One of the significant neutral objectives is known as the Dragon. These are creatures that, when killed, will give the team a small but not insignificant buff. This cumulates when players kill four dragons and gain what is known as the dragon, a large buff that can strongly influence the way that the game goes. The dragon soul is not attainable within 15 minutes, but it is possible to make significant progress towards the dragon soul and gaining the smaller buffs can make a difference. For a more in-depth explanation about the dragons, you can refer to this article: https://win.gg/news/everything-you-need-to-know-about-lols-dragons-and-drakes/

Please note that the article is somewhat out of date as of recent updates, but the premise of what the dragons can give is the same.

#### Rift Heralds Killed
Another of the Neutral monsters available in early game is known as the rift herald. This creature, when killed, gives the killer a temporary ability allowing them to summon the creature to fight for them until killed. When summoned, this creature will charge at nearby towers, dealing massive damage to them and allowign them to put significant pressure on the map. As such, it is very important for early game pressure to get the rift herald if possible. There can only be two rift heralds that spawn in any given game, as its spawn area is taken over by a more powerful monster at the twenty minute mark, but it is marginally possible for both of those to be killed within the first fifteen minutes. For a slightly more in-depth explanation about the rift herald and its uses, you can watch this short video on the topic: https://www.youtube.com/watch?v=ISWm8hNz4Xk


## How are we getting the data?

The data used for this has been collected using the riot api and parsed using custom Python3 code. Due to API limitations, we have maintained the number of matches at 12,000 valid matches. For a further studyt, more matches in more varied ranks could be used, but for now to keep computational burdens relatively low, I have kept at that number. For any interested in looking further at the Riot API itself, here is their developer landing page: https://developer.riotgames.com/

## Program Setup

Below is some setup for the notebook itself. Namely the various libraries that we are importing for use throughout the whole project.

In [33]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from category_encoders import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, plot_roc_curve, roc_auc_score, plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt

## Data Wrangling

*League of Legends* includes over 150 unique champion characters for players to choose from. Were we to encode all of these, we would end up with far too many features to reasonably process. As such, we simplify each of the champions into classes. These classes are provided by the developers themselves and have been parsed into a json file for our use. We load that json file here and set up a function to convert the champion name into its class. Though some champions have a secondary class, most do not and we will not be taking the secondary class into account.

In [2]:
#the dictionary for champion names to their defined classes
with open("champion_classes.json") as file:
    champion_class = json.load(file)

#our transformer for champion names to champion classes
def champ_to_class(champion):
    #special case for Wukong as the API labels it as MonkeyKing
    if champion == "MonkeyKing":
        champ = "wukong"
    
    else:
        champ = champion.lower()
    
    return champion_class[champ]

Our wrangle function sets the match ID as our index, as that is a unique identifier for each match and has no bearing on the match itself. After it has read the dataset in, it takes each of the champion columns and converts it into their class name using the champion name. We do not need to worry about any leaky data as it has been removed as part of the process of fetching the data from the API.

In [3]:
#Our wrangle function to manage our initial data
def wrangle(filepath):
    #read the csv in with Match ID as our index as it is a unique identifier
    df = pd.read_csv(filepath,
                     index_col = "match_id"
                     )
    
    #the columns with champion names
    champion_columns = ["player0_champion", "player1_champion", "player2_champion", "player3_champion", "player4_champion", "player5_champion", "player6_champion", "player7_champion", "player8_champion", "player9_champion"]
    
    #go through each of them to apply the class to the column
    for column in champion_columns:
        df[column] = df[column].apply(champ_to_class)
    
    return df

With that all set up, we can wrangle the data based on our filepath and work from there

In [4]:
#filepath to our data file
filepath = "match_data.csv"

df = wrangle(filepath)

df.head()

,player0_champion,player1_champion,player2_champion,player3_champion,player4_champion,player5_champion,player6_champion,player7_champion,player8_champion,player9_champion,...,red_gold_15,blue_kills_15,red_kills_15,blue_towers_15,red_towers_15,blue_dragons_15,red_dragons_15,blue_heralds_15,red_heralds_15,winner
match_id,,,,,,,,,,,,,,,,,,,,,
NA1_4110079710,Specialist,Slayer,Slayer,Marksman,Tank,Specialist,Fighter,Mage,Marksman,Controller,...,22389,7,4,1,0,1,0,0,1,Red
NA1_4139115929,Fighter,Fighter,Slayer,Marksman,Mage,Tank,Slayer,Mage,Mage,Marksman,...,22045,19,7,1,0,1,0,1,0,Blue
NA1_4139129979,Mage,Marksman,Slayer,Marksman,Controller,Mage,Slayer,Slayer,Marksman,Mage,...,28120,14,18,0,0,0,1,0,0,Blue
NA1_4139074211,Tank,Slayer,Slayer,Marksman,Controller,Tank,Specialist,Mage,Marksman,Controller,...,22890,6,8,0,0,1,0,0,1,Red
NA1_4138986434,Fighter,Fighter,Marksman,Marksman,Mage,Slayer,Fighter,Mage,Marksman,Controller,...,19436,10,2,1,0,0,1,0,0,Blue


## Splitting our Data
We have 12,000 matches available to us. From this, we will be keeping 2000 matches for testing and another 2000 for validation. Our target variable is the winner of the match, as we are looking to predict winners from the early game statistics. As all of our games take place across only two patches with minimal differences and don't have the date data for the matches, we will be randomly splitting the data instead of using datetime.

In [5]:
X = df.drop(columns="winner")
y = df["winner"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=2000, random_state = 709)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state = 709)

## Baseline Accuracy

We are using a classification problem (Picking out our victor). And since we are expecting to have about equal win-rates between each side, it is reasonable to pick an accuracy value. As accuracy is an applicable value, we can select the majority winner and use that as our baseline

In [6]:
baseline_acc = y_train.value_counts(normalize = True).max()

print(f"Our basline accuracy is: {baseline_acc}")

Our basline accuracy is: 0.506875


We have a slight bias towards one side in our data, but not in a way that is unexpected. Though slight, there are a few differences between the sides that coudl weight it to this small of a degree.

## Building our Models
For this project, we will be creating two separate models. The first will be a Logistic Regression Model, while the second will be a tree-based model in a boosting ensemble.

In [7]:
#our logistic regression model; category_encoders currently has a warning for a function used in there own encoder, it is not something handled on this end
model_lr = make_pipeline(
            OneHotEncoder(use_cat_names = True),
            SimpleImputer(),
            LogisticRegression(max_iter=1000, n_jobs =-1)
            )

model_lr.fit(X_train, y_train);

C:\Users\bugab\anaconda3\envs\dataScience\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [8]:
#our tree-based boosting model; same warning for category_encoders as the logistic model
model_boost = make_pipeline(
                    OneHotEncoder(use_cat_names = True),
                    SimpleImputer(),
                    GradientBoostingClassifier(random_state = 709)
                    )

model_boost.fit(X_train, y_train);

C:\Users\bugab\anaconda3\envs\dataScience\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


## Checking our Initial Metrics

With our initial models fitted, we need to check the metrics to see how they are currently performing

In [9]:
print('Logistic Regression Model')
print('Training accuracy:', model_lr.score(X_train, y_train))
print('Validation accuracy:', model_lr.score(X_val, y_val))
print('')
print('Gradient Boosting Model')
print('Training accuracy:', model_boost.score(X_train, y_train))
print('Validation accuracy:', model_boost.score(X_val, y_val))

Logistic Regression Model
Training accuracy: 0.784
Validation accuracy: 0.7815

Gradient Boosting Model
Training accuracy: 0.803625
Validation accuracy: 0.7845


## Tuning our models

Both models performed similarly on the validation set, so we will tune the hyperparameters on both of the models. We will not be using any cross-validation as we have already set a validation set aside

### Logistic Regression Tuning
For the logistic regression tuning, we will be checkign the simpleimputer strategy, the regression solving methods, and various levels of C or penalty strength

In [13]:
lr_param_grid = {
    'simpleimputer__strategy': ['mean', 'median'],
    'logisticregression__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'logisticregression__C': [100.0, 10.0, 1.0, 0.1, 0.01]
}

#setting up our grid search. n_jobs is set to 6 so as to allow my CPU to have some breathing room still
model_gslr = GridSearchCV(
                model_lr,
                param_grid = lr_param_grid,
                n_jobs = -1,
                cv = None,
                verbose=1
                )

model_gslr.fit(X_train, y_train);

Fitting 5 folds for each of 50 candidates, totalling 250 fits


C:\Users\bugab\anaconda3\envs\dataScience\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


### Gradient Boosting Tuning
For tuning our gradient boosting, we will be using learning rate, n_estimators, subsamples, and max depth into account

In [12]:
boost_param_grid = {
    'simpleimputer__strategy': ['mean', 'median'],
    'gradientboostingclassifier__n_estimators': [10, 100, 1000],
    'gradientboostingclassifier__learning_rate': [0.001, 0.01, 0.1],
    'gradientboostingclassifier__subsample': [0.5, 0.7, 1.0],
    'gradientboostingclassifier__max_depth': [3, 7, 9]
}

model_gsboost = GridSearchCV(
                    model_boost,
                    param_grid = boost_param_grid,
                    n_jobs = -1,
                    cv = None,
                    verbose = 1
                    )

model_gsboost.fit(X_train, y_train);

Fitting 5 folds for each of 162 candidates, totalling 810 fits


C:\Users\bugab\anaconda3\envs\dataScience\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


## Checking the tuned metrics
With all of our hyperparameter tuning complete, we can now properly check our metrics

In [14]:
print('Logistic Regression Model')
print('Training accuracy:', model_gslr.score(X_train, y_train))
print('Validation accuracy:', model_gslr.score(X_val, y_val))
print('')
print('Gradient Boosting Model')
print('Training accuracy:', model_gsboost.score(X_train, y_train))
print('Validation accuracy:', model_gsboost.score(X_val, y_val))

Logistic Regression Model
Training accuracy: 0.79075
Validation accuracy: 0.791

Gradient Boosting Model
Training accuracy: 0.804375
Validation accuracy: 0.789


In [36]:
print('Logistic Regression Model performance on testing set: ', model_gslr.score(X_test, y_test))

Logistic Regression Model performance on testing set:  0.76


## Results and Summary
After our hyperparameter tuning, our best accuracy available to us is the linear regression model with a 79.1% accuracy on our validation set, and 76% accuracy on our test set. Overall, this is much higher than our baseline, and a good launchpad for further research. Though not the most accurate, it does show that early game statistics will show a prevalence towards the victor of the game overall.

## Limitations and further study
Though enlightening to see the impact of our data, there is still some limitations that must be taken into account. Namely, the narrow scope of data collected. Though we had a dataset of 12,000 matches, they were all taken from only a few ranks and tiers of the competitive ladder. Were a model truly able to generalize to all players, it would need to take data from all the ranks sufficiently in order to ensure that rank does not play a significant factor in other parts of the decision.

A few of our other limitations is in communication of results. Unfortunately, with the current models, properly seeing the effect of the individual attributes is quite difficult to summarize. Not only will they need scaling, but also condensing in order to truly understand the differences between players having a specific value and having an overall effect based upon a given decision.

Lastly, is the need for further tuning and usage of other potential methods. Though a gradient boost was used for our tree-based option, it may vbe prevalent to explore other tree-based options to see if they yield more accuracy in the long-run. Overall, this is a topic that will require further study before being able to properly examine everything.